In [1]:
import json

In [2]:
f = open("US_category_id.json",)

In [3]:
data = json.load(f)

In [4]:
categoryMap = {}
for i in data["items"]:
    categoryMap[i["id"]]  = i["snippet"]["title"]

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('USvideos.csv', usecols=['title','title','channel_title', 'tags', 'description','category_id'])

In [7]:
num_rows = df.count()[0]

In [10]:
videoData = []
for i in range(num_rows):
    current_vid = df.iloc[i]
    row = []
    channel_title = preProcess(current_vid["channel_title"], 0)
    title = preProcess(current_vid["title"], 4)
    tags = preProcess(current_vid["tags"], 4)
    description = preProcess(current_vid["description"], 4)
    category = categoryMap[str(current_vid["category_id"])]
    
    row.append(channel_title)
    row.append(title)
    row.append(tags)
    row.append(description)
    row.append(category)
    videoData.append(row)


In [9]:
def preProcess(data, length):
    dataArr = str(data).strip().split(" ")
    for i in  dataArr:
        if len(i) < length:
            dataArr.remove(i)
    dataArr = ";".join(dataArr)
    return dataArr

In [135]:
import csv  

header = ['Video Title', 'Channel Title', 'Tags', 'Description', 'Category']

with open('video_data.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    writer.writerows(videoData)